In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import os
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def preprocess_java_file(file_path):
    with open(file_path, 'r') as file:
        java_code = file.read()
    # Tokenize the Java code
    tokens = word_tokenize(java_code)
    return tokens

# Function to iterate over Java files in a directory
def iterate_java_files(directory, label):
    for root, _, files in os.walk(directory):
        for file in files:
            yield TaggedDocument(words=preprocess_java_file(os.path.join(root, file)), tags=[label])

# Dataset

In [ ]:
benign_java = "FINALbenignjava/"
malware_java = "FINALmalwarejava/"
benign_files = list(iterate_java_files(benign_java, 'benign'))
malware_files = list(iterate_java_files(malware_java, 'malware'))
all_files = benign_files + malware_files
print(len(all_files))

# Combine labels for the files: 0 for benign, 1 for malware
labels = [0] * len(benign_files) + [1] * len(malware_files)

# Split the data into train and test sets
train_files, test_files, train_labels, test_labels = train_test_split(all_files, labels, test_size=0.2, random_state=42)

print("Number of training samples:", len(train_files))
print("Number of test samples:", len(test_files))

499
Number of training samples: 399
Number of test samples: 100


In [ ]:
print(labels[90:110])
print(len(malware_files))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
199


In [ ]:
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(train_files)
model.train(train_files, total_examples=model.corpus_count, epochs=model.epochs)

# Save the model
model.save("doc2vec_model")

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load the saved Doc2Vec model
model = Doc2Vec.load("doc2vec_model")

# Infer document vectors for the training and test data
train_vectors = [model.infer_vector(doc.words) for doc in train_files]
test_vectors = [model.infer_vector(doc.words) for doc in test_files]

# Convert the list of vectors to numpy arrays
train_vectors = np.array(train_vectors)
test_vectors = np.array(test_vectors)

# Extract labels from TaggedDocument objects
train_labels = [doc.tags[0] for doc in train_files]
test_labels = [doc.tags[0] for doc in test_files]

# Train a classifier, for example, Logistic Regression
classifier = LogisticRegression()
classifier.fit(train_vectors, train_labels)
# Predict labels for the test data
predictions = classifier.predict(test_vectors)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-c28400fe38a6>", line 10, in <cell line: 10>
    train_vectors = [model.infer_vector(doc.words) for doc in train_files]
  File "<ipython-input-27-c28400fe38a6>", line 10, in <listcomp>
    train_vectors = [model.infer_vector(doc.words) for doc in train_files]
  File "/usr/local/lib/python3.10/dist-packages/gensim/models/doc2vec.py", line 651, in infer_vector
    train_document_dm(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, ano

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(test_labels)
print(test_vectors)

[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1]


NameError: name 'test_vectors' is not defined

## Export vectors (single vector / file)

In [ ]:
from gensim.models import Doc2Vec
import os

# Load the pre-trained model
model_path = 'doc2vec_model'  # Specify the path to your model
model = Doc2Vec.load(model_path)

# Define the paths
java_files_path = 'FINALmalwarejava'  # Specify the folder path where the Java files are located
output_vectors_path = 'FINALmalwarevec'  # Specify the folder path where the vectors should be saved
testt = []

# Function to infer vector from Java file and save it
def infer_and_save_vector(file_path, model, output_path):
    # Read the content of the Java file
    with open(file_path, 'r') as f:
        java_code = f.read()

    # Infer the vector from the Java code
    inferred_vector = model.infer_vector(java_code.split())

    # Extract the file name without extension to use as the output file name
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # Define the output file path for the inferred vector
    output_file_path = os.path.join(output_path, f'vec_{file_name}.txt')

    # Save the inferred vector to a text file
    with open(output_file_path, 'w') as output_file:
        # Join the vector elements with a comma and write it to the file
        vector_as_string = ','.join(map(str, inferred_vector))
        testt.append(inferred_vector)
        output_file.write(str(inferred_vector))

# Iterate through each Java file in the specified directory
for java_file in os.listdir(java_files_path):
    # Construct the full file path of the Java file
    file_path = os.path.join(java_files_path, java_file)

    # Infer and save the vector
    infer_and_save_vector(file_path, model, output_vectors_path)

predictions = classifier.predict(testt)
print(predictions)
print(f"All vectors have been saved to {output_vectors_path}.")


NameError: name 'classifier' is not defined

In [ ]:
import shutil

# Specify the source folder and output zip file
folder = 'benign_vec'  # Replace with the path and name of the zip file (without .zip)

# Create a zip file from the folder
shutil.make_archive(folder, 'zip', folder)

print(f'Folder {folder} has been zipped to {folder}.zip')

Folder benign_vec has been zipped to benign_vec.zip


Infer to vector

In [ ]:
print(Doc2Vec)

<class 'gensim.models.doc2vec.Doc2Vec'>


In [ ]:
java_benign_folder = 'FINALbenignjava'

# Load the saved Doc2Vec model
model = Doc2Vec.load("doc2vec_model")

In [ ]:
def split_functions(file_path):
    functions = []  # List to hold the functions
    current_function = []  # List to hold lines of the current function
    inside_function = False

    # Access modifier keywords in Java
    access_modifiers = ["public", "private", "protected"]

    # Open the file for reading
    with open(file_path, 'r') as file:
        for line in file:
            # Strip whitespace from the start and end of the line
            stripped_line = line.strip()

            # Determine if the line starts with a function definition
            # Check if the line starts with one of the access modifiers and contains "static"
            # OR if the line starts with a type (e.g., `String`, `int`, `void`) and contains "static"
            if any(stripped_line.startswith(modifier) for modifier in access_modifiers) and "static" in stripped_line:
                # If already inside a function, save the previous function and reset current_function
                if inside_function:
                    functions.append("\n".join(current_function))
                    current_function = []

                # Start a new function
                inside_function = True
                current_function.append(stripped_line)
            elif stripped_line.startswith(("String", "int", "long", "double", "float", "boolean", "char", "void")) and "static" in stripped_line:
                # For default access level, check if the line starts with a type and contains "static"
                if inside_function:
                    functions.append("\n".join(current_function))
                    current_function = []
                inside_function = True
                current_function.append(stripped_line)
            elif inside_function:
                # If we are inside a function, add the line to the current function
                current_function.append(stripped_line)

    # Append the last function to the list if one is still being tracked
    if current_function:
        functions.append("\n".join(current_function))

    return functions


In [ ]:

# # Example usage:
# file_path = "/content/FINALbenignjava/00705D578467B6562DF34506BE1213182024775059D2B7959644AC48719E8CC6.txt"  # Replace with the path to your text file
# functions_array = split_functions(file_path)
# print(len(functions_array))

# # Output the functions array to verify
# # for i, function in enumerate(functions_array):
# #     print(f"Function {i + 1}:\n{function}\n")

1649


In [ ]:
# import networkx as nx

# filepath = '/content/00705D578467B6562DF34506BE1213182024775059D2B7959644AC48719E8CC6.gexf'
# def count_nodes_in_gexf(filepath):
#     # Load the .gexf file
#     graph = nx.read_gexf(filepath)

#     num_nodes = graph.number_of_nodes()
#     print(f"Number of nodes: {num_nodes}")
#     node_names = [node for node in graph.nodes()]
#     node_vectors = ["123"+node for node in node_names]

#     print(len(node_names))
#     print(f"Names of the nodes: {node_names}")
#     print(f"Names of the nodes: {node_vectors}")

# # Example usage:
# count_nodes_in_gexf(filepath)


Number of nodes: 4796
4796
Names of the nodes: ['<init>', 'equals', 'getKey', 'getValue', 'setValue', 'toString', 'append', 'supportRemove', 'access$100', 'descendingIterator', 'put', 'valueOf', 'eldest', 'hasNext', 'size', 'iterator', 'next', 'get', 'iteratorWithAdditions', 'newest', 'putIfAbsent', 'remove', 'isEmpty', 'keySet', '<clinit>', 'values', 'clone', 'isAtLeast', 'compareTo', 'addObserver', 'getCurrentState', 'removeObserver', 'getLifecycle', 'ordinal', 'getCallback', 'dispatchEvent', 'getStateAfter', 'onStateChanged', 'min', 'getDeclaredConstructor', 'getAdapterName', 'replace', 'setAccessible', 'newInstance', 'getClass', 'isAccessible', 'getGeneratedAdapterConstructor', 'getName', 'getPackage', 'forName', 'length', 'getCanonicalName', 'substring', 'getSuperclass', 'value', 'entrySet', 'add', 'hashCode', 'onCreate', 'onResume', 'onStart', 'dispatch', 'getActivity', 'handleLifecycleEvent', 'dispatchCreate', 'dispatchResume', 'dispatchStart', 'findFragmentByTag', 'getFragmentM

In [ ]:
import re
import networkx as nx


apk_id = '0107D3360DF1C04AF875C349021EA08A4F76AFCABF85884A65A64A4A3943AE35'
gexf_file = f'/content/{apk_id}.gexf'
java_file_path = f'/content/FINALbenignjava/{apk_id}.txt'

# Load the .gexf file
graph = nx.read_gexf(gexf_file)

num_nodes = graph.number_of_nodes()
print(f"Number of nodes: {num_nodes}")
node_names = [node for node in graph.nodes()]
node_vectors = ["123"+node for node in node_names]

print(len(node_names))
print(f"Names of the nodes: {node_names}")
print(f"Names of the nodes: {node_vectors}")


# Read Java file
def read_java_file(filepath):
    with open(filepath, 'r') as file:
        content = file.read()
    return content

# Extract function content
def extract_function_content(java_content, function_name):
    # Use a regular expression to match function definition and its contents
    pattern = rf"\b{function_name}\b\s*\(([^)]*)\)\s*{{(.*?)}}"  # Capture function name, parameters, and body
    match = re.search(pattern, java_content, re.DOTALL)

    if match:
        # Extract function parameters and body
        parameters = match.group(1)
        body = match.group(2)
        return parameters, body
    else:
        return None, None

# List of Java function names you want to search
function_names = node_names

# Read the content of the Java file
java_content = read_java_file(java_file_path)
fc_str = []

# Iterate through the list of function names
for function_name in function_names:
    parameters, body = extract_function_content(java_content, function_name)
    if parameters is not None and body is not None:
        processed_fc = f'{function_name}({parameters})'+'{'f'{str(body)}''}'
        # print(f'{function_name}({parameters})', end = "")
        # print('{'f'{str(body)}''}')
        # print('{'f'Body: {len(body)}''}')
    else:
        processed_fc = f'{function_name}({parameters})'+"{}"
    # print(processed_fc)
    fc_str.append(processed_fc)

Number of nodes: 902
902
Names of the nodes: ['<init>', 'execute', 'e', 'c', 'a', 'newThread', 'format', 'valueOf', 'setName', 'getAndIncrement', 'b', '<clinit>', 'filter', 'd', 'onClick', 'onFocusChange', 'f', 'instantiateActivity', 'instantiateApplication', 'instantiateProvider', 'instantiateReceiver', 'instantiateService', 'read', 'v', 'r', 'l', 'h', 'write', 'z', 'M', 'D', 'x', 'H', 'startForegroundService', 'startService', 'j', 'p', 't', 'J', 'B', 'F', 'ordinal', 'values', 'run', 'i', 'append', 'toString', 'next', 'getValue', 'hasNext', 'g', 'clone', 'compareTo', 'postAtFrontOfQueue', 'intValue', 'remove', 'onCallbackDied', 'onBind', 'contains', 'O0', 'L1', 'e0', 'l0', 'newSetFromMap', 'getInt', 'close', 'add', 'isEmpty', 'moveToNext', 'iterator', 'unlock', 'lock', 'k', 'compareAndSet', 'fill', 'copyOf', 'unmodifiableSet', 'size', 'equalsIgnoreCase', 'toLowerCase', 'put', 'get', 'entrySet', 'containsKey', 'getKey', 'toArray', 'addAll', 'setLength', 'o', 'm', 'w', 's', 'onServiceCo

In [ ]:
print(len(fc_str))
print(fc_str)

902
['<init>(None){}', 'execute(Runnable p2){\n        a.a.e().c(p2);\n        return;\n    }', 'e(com.simplemobileapps.literstomilliliters.MainActivity p1){\n        this.a = p1;\n        return;\n    }', 'c(){\n        return;\n    }', 'a(){\n        return;\n    }', 'newThread(Runnable p4){\n        Thread v0_1 = new Thread(p4);\n        String v4_3 = new Object[1];\n        v4_3[0] = Integer.valueOf(this.a.getAndIncrement());\n        v0_1.setName(String.format(arch_disk_io_%d, v4_3));\n        return v0_1;\n    }', 'format(None){}', 'valueOf(String p1){\n        return ((androidx.lifecycle.c$a) Enum.valueOf(androidx.lifecycle.c$a, p1));\n    }', 'setName(None){}', 'getAndIncrement(None){}', 'b(){\n        return;\n    }', '<clinit>(None){}', 'filter(CharSequence p1, int p2, int p3, android.text.Spanned p4, int p5, int p6){\n        return com.simplemobileapps.literstomilliliters.MainActivity.d(p1, p2, p3, p4, p5, p6);\n    }', 'd(com.simplemobileapps.literstomilliliters.MainActivi

In [ ]:
model = Doc2Vec.load('doc2vec_model')

# Preprocess if necessary (example with NLTK)
from nltk import word_tokenize

preprocessed_strings = []
for string in fc_str:
    tokens = word_tokenize(string)
    preprocessed_strings.append(tokens)  # Adjust based on your preprocessing needs

# Embed each string (or preprocessed list)
string_vectors = []
for string in preprocessed_strings:  # Use fc_str directly if not preprocessing
    vector = model.infer_vector(string)  # Use infer_vector for unseen data
    string_vectors.append(vector)

print(string_vectors[0])
print(len(string_vectors))

[-0.17078796  0.01895706  0.15993723 -0.14146324 -0.04932907  0.13195829
 -0.07906992  0.18960942  0.08446534 -0.04470031 -0.13442247  0.04974963
 -0.09865545 -0.18816675  0.04527828  0.09255666 -0.2133767   0.02500367
  0.0221045   0.25650862 -0.05460303 -0.06693719  0.24956277 -0.01993687
 -0.1254656  -0.02142902  0.02049052  0.14547539 -0.04266426  0.03454472
 -0.12280924  0.2620987   0.17936185  0.38377768 -0.02514891 -0.07637109
 -0.26081342 -0.03783749  0.17639288  0.11737633  0.13953683  0.4099621
  0.07923328  0.0836129  -0.02896654  0.0276887   0.24178493 -0.04883606
 -0.08029555 -0.04495333  0.08390194  0.12310306 -0.0061696   0.10303362
  0.10899989 -0.51403964  0.0863261  -0.04260461  0.17754756 -0.20161504
  0.18133755 -0.0913479  -0.19161217  0.14868277 -0.1845741   0.02858944
  0.03490526 -0.06067345  0.115834    0.34554046  0.0782137   0.26099345
 -0.01985271 -0.03571878  0.02618442  0.03563509  0.10225347 -0.02481369
 -0.18826231  0.10286552 -0.20937656 -0.1935003   0.

In [ ]:
import os
import re
import networkx as nx
from gensim.models import Doc2Vec

def extract_and_embed_functions(gexf_folder, java_folder, doc2vec_model_path, output_folder):
  """
  Iterates over GEXF files in a folder, extracts function content from corresponding Java files,
  embeds them using a Doc2Vec model, and saves the results to text files.

  Args:
      gexf_folder (str): Path to the folder containing GEXF files.
      java_folder (str): Path to the folder containing corresponding Java files.
      doc2vec_model_path (str): Path to the pre-trained Doc2Vec model.
      output_folder (str): Path to the folder where output text files will be saved.
  """

  # Load the Doc2Vec model
  model = Doc2Vec.load(doc2vec_model_path)

  # Iterate over GEXF files
  for gexf_filename in os.listdir(gexf_folder):
    if gexf_filename.endswith('.gexf'):
      apk_id = gexf_filename[:-5]  # Extract APK ID from filename (assuming format)

      # Construct file paths
      gexf_filepath = os.path.join(gexf_folder, gexf_filename)
      java_filepath = os.path.join(java_folder, f'{apk_id}.txt')
      output_filepath = os.path.join(output_folder, f'{apk_id}.txt')

      # Load the GEXF graph
      graph = nx.read_gexf(gexf_filepath)

      # Extract function names from nodes
      function_names = [node for node in graph.nodes()]

      # Read Java content
      try:
          with open(java_filepath, 'r') as file:
              java_content = file.read()
      except FileNotFoundError:
          print(f"Java file not found: {java_filepath}")
          continue  # Skip to next iteration if Java file is missing

      # Extract and embed function content
      fc_str = []
      for function_name in function_names:
          parameters, body = extract_function_content(java_content, function_name)
          if parameters is not None and body is not None:
              processed_fc = f'{function_name}({parameters})' + '{' + f'{body}' + '}'
          else:
              processed_fc = f'{function_name}({parameters})' + '{}'
          fc_str.append(processed_fc)

      # Preprocess if necessary (using NLTK for example)
      from nltk import word_tokenize  # Import NLTK library for tokenization

      preprocessed_strings = []
      for string in fc_str:
          tokens = word_tokenize(string)
          preprocessed_strings.append(tokens)

      # Embed each string (using preprocessed strings)
      string_vectors = []
      for string in preprocessed_strings:
          vector = model.infer_vector(string)  # Use infer_vector for unseen data
          string_vectors.append(vector)

      # Save string vectors to a text file (one vector per line)
      try:
          with open(output_filepath, 'w') as output_file:
              for vector in string_vectors:
                  output_file.write(str(vector) + '\n')  # Write each vector on a new line
      except IOError:
          print(f"Error saving output to: {output_filepath}")

# Example usage (assuming you have set up the necessary folders and model)
gexf_folder = 'Xgexf'
java_folder = 'Xjava'
doc2vec_model_path = 'doc2vec_model'
output_folder = 'benign_vec'
extract_and_embed_functions(gexf_folder, java_folder, doc2vec_model_path, output_folder)

KeyboardInterrupt: 

[array([-0.17078796,  0.01895706,  0.15993723, -0.14146324, -0.04932907,
        0.13195829, -0.07906992,  0.18960942,  0.08446534, -0.04470031,
       -0.13442247,  0.04974963, -0.09865545, -0.18816675,  0.04527828,
        0.09255666, -0.2133767 ,  0.02500367,  0.0221045 ,  0.25650862,
       -0.05460303, -0.06693719,  0.24956277, -0.01993687, -0.1254656 ,
       -0.02142902,  0.02049052,  0.14547539, -0.04266426,  0.03454472,
       -0.12280924,  0.2620987 ,  0.17936185,  0.38377768, -0.02514891,
       -0.07637109, -0.26081342, -0.03783749,  0.17639288,  0.11737633,
        0.13953683,  0.4099621 ,  0.07923328,  0.0836129 , -0.02896654,
        0.0276887 ,  0.24178493, -0.04883606, -0.08029555, -0.04495333,
        0.08390194,  0.12310306, -0.0061696 ,  0.10303362,  0.10899989,
       -0.51403964,  0.0863261 , -0.04260461,  0.17754756, -0.20161504,
        0.18133755, -0.0913479 , -0.19161217,  0.14868277, -0.1845741 ,
        0.02858944,  0.03490526, -0.06067345,  0.115834  ,  0.3

In [ ]:
import os

def count_files(folder_path):
  """Counts the number of files in a folder (excluding subfolders).

  Args:
      folder_path (str): Path to the folder.

  Returns:
      int: The number of files in the folder.
  """

  try:
    # Use os.scandir to get directory entries without opening files
    entries = os.scandir(folder_path)
    # Filter for files (not directories)
    files = [entry for entry in entries if entry.is_file()]
    return len(files)
  except FileNotFoundError:
    print(f"Folder not found: {folder_path}")
    return 0  # Return 0 if folder doesn't exist

# Example usage (replace with your actual folder path)
folder_path = 'benign_vec'
num_files = count_files(folder_path)
print(f"Number of files in '{folder_path}': {num_files}")


Number of files in 'benign_vec': 73
